In [ ]:

import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import matplotlib.pylab as plt
import os
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, ConvLSTM2D, TimeDistributed, Bidirectional, LSTM
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobilenet_v2_preprocess_input
from keras.utils import Sequence
from tensorflow.keras.callbacks import Callback


class DataGen(Sequence):
    """ A sequence of data for training/test/validation, loaded from memory
    batch by batch. Extends the tensorflow.keras.utils.Sequence: https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

    Attributes
    ----------
    base_path : str
                path to the folder including the samples.
    filenames : list<str>
                list of sample filenames.
    labels : list<str>
             list of sample labels.
    batch_size : int
                 batch size to load samples

    """

    def __init__(self, base_path, filenames, labels, batch_size, Preprocess_input):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.Preprocess_input = Preprocess_input

    def __len__(self):
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        return np.array([self.Preprocess_input(np.load(os.path.join(self.base_path, file_name))) for file_name in batch_x]), np.array(batch_y)

def GetPretrainedModel(ModelConstructor, input_shape=(224, 224, 3), print_summary=True):
    """ Builds the VGG16 2D CNN with the Imagenet weights, freezing all layers except layers_to_finetune

    Parameters
    ----------
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    print_summary : bool
                    If True prints the model summary.

    Returns
    -------
    model : Sequential
          The instantiated model.
    """

    model = ModelConstructor(include_top=False, weights="imagenet", input_shape=input_shape)

    for layer in model.layers:
        layer.trainable = False

    return model

def getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    #model.add(LSTM(units=128, return_sequences=False))

    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def getConvLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# definitions of two end-to-end models + definitions of experiments
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import matplotlib.pylab as plt
import os
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, ConvLSTM2D, TimeDistributed, Bidirectional, LSTM
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobilenet_v2_preprocess_input
from keras.utils import Sequence

class DataGen(Sequence):
    """ A sequence of data for training/test/validation, loaded from memory
    batch by batch. Extends the tensorflow.keras.utils.Sequence: https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

    Attributes
    ----------
    base_path : str
                path to the folder including the samples.
    filenames : list<str>
                list of sample filenames.
    labels : list<str>
             list of sample labels.
    batch_size : int
                 batch size to load samples

    """

    def __init__(self, base_path, filenames, labels, batch_size, Preprocess_input):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.Preprocess_input = Preprocess_input

    def __len__(self):
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        return np.array([self.Preprocess_input(np.load(os.path.join(self.base_path, file_name))) for file_name in batch_x]), np.array(batch_y)

def GetPretrainedModel(ModelConstructor, input_shape=(224, 224, 3), print_summary=True):
    """ Builds the VGG16 2D CNN with the Imagenet weights, freezing all layers except layers_to_finetune

    Parameters
    ----------
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    print_summary : bool
                    If True prints the model summary.

    Returns
    -------
    model : Sequential
          The instantiated model.
    """

    model = ModelConstructor(include_top=False, weights="imagenet", input_shape=input_shape)

    for layer in model.layers:
        layer.trainable = False

    return model

def getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    #model.add(LSTM(units=128, return_sequences=False))

    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def getConvLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

def runEndToEndExperiment(getLSTMModel, getConvModel, ModelConstructor, pretrained_input_shape, Preprocess_input, batchSize, datasetBasePath, npyBasePath, featuresPath, samplesMMapName, lablesMMapName, endToEndModelName, rState, savePath):
        return  getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape)




class InferenceTimeCallback(Callback):
    def on_predict_batch_begin(self, batch, logs=None):
        self.start_time = time.time()

    def on_predict_batch_end(self, batch, logs=None):
        self.end_time = time.time()
        self.inference_time = self.end_time - self.start_time
        print(f"Inference time for batch {batch}: {self.inference_time} seconds")




In [ ]:
!pip install codecarbon

### **Test sui tempi di inferenza per l'ultimo split sul modello ConvLSTM**



*   Accuratezza: 95 %
*   Curva AUC: 99.4 %
*   Peso modello: 25.5 MB
*   Impronta di carbonio:   Kg/C02
*   Consumo Energetico medio per 102 secondi di video:  0.0018 kW/h
*   Tempo medio di inferenza per batch:  0.43  s



In [ ]:
# Monto il drive Google
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:


from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_v3_preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


MobileNetV3Small_ConvLSTM = runEndToEndExperiment(
    getConvLSTMModel,
    GetPretrainedModel,
    MobileNetV3Small,
    (224, 224, 3),
    mobilenet_v3_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + ConvLSTM',
    42,
    '/content/gdrive/ My Drive/MobileNetV3Small_ConvLSTM'
)


MobileNetV3Small_ConvLSTM.load_weights('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/ConvLSTM/final_model_fold_5.h5')



In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import time


from codecarbon import track_emissions


def load_keras_model(model_path):
    model = load_model(model_path)
    return model

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')  # / 255.0
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:

        batches = []

    return batches


def run_inference(model, input_data):
    output_data = model.predict(input_data)
    return output_data

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels

@track_emissions(project_name="MV3_Small_Inference_ConvLSTM")
def inferenceV3_ConvLSTM():
    model_path = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/ConvLSTM/final_model_fold_5.h5'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    model = load_keras_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    total_emissions = 0
    total_energy_consumed = 0
    y_probs = []
    for video in all_videos:
        start_time = time.time()
        output_data = run_inference(model, video)
        end_time = time.time()
        inference_time = end_time - start_time
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time_per_batch = total_inference_time / len(all_videos)


    average_emissions_per_video = total_emissions / len(all_videos)


    average_energy_per_batch = total_energy_consumed / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time per Batch: {average_inference_time_per_batch:.6f} seconds')



    print('Classification Report:')
    print(classification_report(all_labels, y_pred, target_names=['NonViolent', 'Violent']))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm')

if __name__ == '__main__':
    inferenceV3_ConvLSTM()


### **Test sui tempi di inferenza per l'ultimo split sul modello BiLSTM**



*   Accuratezza: 90.6 %
*   Curva AUC:  97.84 %
*   Peso modello: 336.6 MB
*   Consumo Energetico medio per 102 secondi di video:   0.0022 kw/h
*   Tempo medio di inferenza per batch: 0.7   s

In [ ]:


from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_v3_preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback


MobileNetV3Small_BiLSTM = runEndToEndExperiment(
    getLSTMModel,
    GetPretrainedModel,
    MobileNetV3Small,
    (224, 224, 3),
    mobilenet_v3_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + BiLSTM',
    42,
    '/content/gdrive/ My Drive/MobileNetV3Small_BiLSTM'
)


MobileNetV3Small_BiLSTM.load_weights('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/BiLSTM/final_model_fold_5.h5')



In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import time



def load_keras_model(model_path):
    model = load_model(model_path)
    return model

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32') #/ 255.0
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:

        batches = []

    return batches

def run_inference(model, input_data):
    output_data = model.predict(input_data)
    return output_data

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_BiLSTM")
def inferenceV3_BiLSTM():
    model_path = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/BiLSTM/final_model_fold_5.h5'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    model = load_keras_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        start_time = time.time()
        output_data = run_inference(model, video)
        end_time = time.time()
        inference_time = end_time - start_time
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time_per_batch = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time per Batch: {average_inference_time_per_batch:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred, target_names=['NonViolent', 'Violent']))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm')

if __name__ == '__main__':
    inferenceV3_BiLSTM()